#  regridding open RiOMar/GAMAR via Kerchunk → write a HEALPIX Zarr

This notebook make create HEALPIxZARR of ROI> 

It opens the dataset from a **Kerchunk catalog** generated on HPC, and the *same* workflow can open:
- directly on the HPC filesystem, or
- over HTTPS via the DATARMOR export service (`https://data-fair2adapt.ifremer.fr/`).

## Goal
Produce a temporary Zarr (e.g. `small.zarr`) that is then used as input to:
- `regrid_apply_ufunc.ipynb`

## Outputs
- `OUT_ZARR`: a lightweight subset written to Zarr (local or HPC scratch)

## Steps
1. **Open the dataset** from a Kerchunk catalog (portable: HPC filesystem or HTTPS export)
2. **Select variables** and ensure lon/lat are explicit coordinates
3. **Build a polygon mask** helper for the model grid
4. **Define the ROI** and apply the mask (load ROI polygon, mask, and subset)
5. **regrid** 
6. **Write a  Zarr** 




## Kerchunk catalog options

A **Kerchunk catalog** is a JSON mapping that lets Xarray open a collection of NetCDF (or similar) files as a *virtual* Zarr dataset.  
Depending on where you run this notebook, you can point to:

- **HPC filesystem path** (fast, when you have direct access to `/scale/...`)
- **HTTPS export** (portable, when you access the same data through `https://data-fair2adapt.ifremer.fr/`)

Below are example catalog paths that have been created previously (kept here as a reference).


## 1. **Open the dataset** from a Kerchunk catalog (portable: HPC filesystem or HTTPS export)

Many Kerchunk catalogs store references to the original files (often as absolute HPC paths).  
When opening through HTTPS, we **rewrite** those references from:

`/scale/project/lops-oh-fair2adapt/...` → `https://data-fair2adapt.ifremer.fr/...`

The cell below:
1. Detects whether the HPC path exists (so we are running on the cluster).
2. Otherwise loads the JSON over HTTPS, patches references in-memory, and opens the dataset.
3. (Optional) can cache the patched references locally as a parquet file for faster re-opening.


In [1]:
%%time
# this takes 4 s
#
import json
import fsspec
from pathlib import Path
import os
import psutil
from dask.distributed import Client, LocalCluster
import xarray as xr
import numpy as np

import xdggs
import healpix_geo

time_chunk_size = 24  # 1 day as a chunk
child_level=13

with np.load("parent_ids.npz") as data:
    parent_ids = data["parent_ids"]
    parent_level = int(data["parent_level"])

    

HPC_PREFIX    = "/scale/project/lops-oh-fair2adapt/"
HTTPS_PREFIX  = "https://data-fair2adapt.ifremer.fr/"
CATALOG_PATH  = "fpaul/tmp/riomar_3months.json"
#CATALOG_PATH  = "riomar-virtualizarr/Y2023.json"
CATALOG_PATH  = "riomar-virtualizarr/YALL.json"
OUT_PARQUET   = "riomar_3months_.parq"   # local parquet refs cache
OUT_ZARR   = "riomar-zarr_tina/Y2023.zarr"  # local parquet refs cache


def patch_kc_refs_inplace(kc, hpc_prefix=HPC_PREFIX, https_prefix=HTTPS_PREFIX):
    refs = kc.get("refs", kc.get("references"))
    if refs is None:
        raise KeyError("Can't find 'refs' (or 'references') in kerchunk JSON")

    def patch_target(x):
        if isinstance(x, str) and x.startswith(hpc_prefix):
            return https_prefix + x[len(hpc_prefix):]
        return x

    for k, v in list(refs.items()):
        if isinstance(v, list) and v and isinstance(v[0], str):
            refs[k] = [patch_target(v[0])] + v[1:]
        elif isinstance(v, str):
            refs[k] = patch_target(v)

    kc["refs"] = refs
    return kc


# ------------------------------
# 1) HPC mode: open directly
# ------------------------------
if Path(HPC_PREFIX).exists():
    ##on HPC


    zarr_hp_file_path = HPC_PREFIX + OUT_ZARR

    # pick a fast local path on the compute node
    local_dir = (
     "/tmp"
    )
    local_dir = str(Path(local_dir) / "dask-scratch")
    print("Using Dask local_directory:", local_dir)
    
    
    print("=== Starting local Dask cluster (auto-sized) ===")
    
    cpu = os.cpu_count() or 1
    total_gb = psutil.virtual_memory().total / (1024**3)
    
    # Good “use most, but not all” defaults:
    n_workers = cpu                   # ~1 worker per CPU core
    threads_per_worker = 1            # best for numpy-heavy compute
    memory_limit_gb = (total_gb * 0.85) / n_workers  # leave ~15% headroom
    memory_limit = f"{memory_limit_gb:.2f}GB"
    n_workers=32
    cluster = LocalCluster(
        n_workers=n_workers,
    #    threads_per_worker=threads_per_worker,
        processes=True,
        memory_limit=memory_limit,
        local_directory=local_dir,   # <--- THIS FIXES THE WARNING
        dashboard_address=":8787",
    )
    client = Client(cluster)
    
    print("Dask dashboard:", client.dashboard_link)
    
    print("\n=== Dask cluster resources ===")
    info = client.scheduler_info()
    workers = info["workers"]
    
    total_threads = sum(w["nthreads"] for w in workers.values())
    total_mem_gb = sum(w["memory_limit"] for w in workers.values()) / (1024**3)
    
    print(f"Workers: {len(workers)}")
    print(f"Total threads: {total_threads}")
    print(f"Total memory limit: {total_mem_gb:.2f} GB")
    
    # Optional: per-worker details
    #for addr, w in workers.items():
     #   print(f"- {addr}: nthreads={w['nthreads']}, mem_limit={w['memory_limit']/1e9:.2f} GB »)
    KERCHUNK_CATALOG = HPC_PREFIX + CATALOG_PATH
    print("Running in HPC mode:", KERCHUNK_CATALOG)

    ds = xr.open_dataset(KERCHUNK_CATALOG, engine="kerchunk", chunks={})

# ------------------------------
# 2) HTTPS mode: prefer local parquet cache if present
# ------------------------------
else:
    cluster = LocalCluster()
    client = Client(cluster)
    client
    zarr_hp_file_path =  OUT_ZARR

    KERCHUNK_CATALOG = HTTPS_PREFIX + CATALOG_PATH
    print("Running in HTTPS mode:", KERCHUNK_CATALOG)
    # If parquet refs already exist locally, open them (fast path)
    # This part is commented since on the fly transformation is faster than loading the parquet file in actual config
    # (check why at some point) 
    # Loading from local parquet is also slower than loading json and convert the path on the fly...
    # thus i deactivate the if here
    #if Path(OUT_PARQUET).exists():
    if False and Path(OUT_PARQUET).exists():
        print(f"✅ Found local parquet refs: ./{OUT_PARQUET} -> opening that")
        xr.open_dataset(OUT_PARQUET, engine="kerchunk", chunks={})

    # Else: fetch JSON, patch refs to https, open, AND write parquet refs cache
    else:
        print(f"ℹ️ No local parquet refs found at ./{OUT_PARQUET} -> creating them from JSON")

        with fsspec.open(KERCHUNK_CATALOG, "rt") as f:
            kc = json.load(f)

        kc = patch_kc_refs_inplace(kc)

        # open now (from in-memory dict)
        ds = xr.open_dataset(kc, engine="kerchunk", chunks={})

        ## write parquet refs cache for next time
        #import kerchunk.df as kcdf
        #kcdf.refs_to_dataframe(kc, OUT_PARQUET)
        #print("✅ Wrote kerchunk parquet refs to:", OUT_PARQUET)

ds

Running in HTTPS mode: https://data-fair2adapt.ifremer.fr/riomar-virtualizarr/YALL.json
ℹ️ No local parquet refs found at ./riomar_3months_.parq -> creating them from JSON
CPU times: user 47.1 s, sys: 13.3 s, total: 1min
Wall time: 1min 26s


<xarray.Dataset> Size: 79TB
Dimensions:              (time_counter: 201600, s_rho: 40, s_w: 41, y_rho: 838,
                          x_rho: 727, y_u: 838, x_u: 726, y_v: 837, x_v: 727,
                          axis_nbounds: 2)
Coordinates: (12/19)
  * time_counter         (time_counter) datetime64[ns] 2MB 2020-02-01T00:52:3...
  * s_rho                (s_rho) float32 160B -0.9875 -0.9625 ... -0.0125
  * s_w                  (s_w) float32 164B -1.0 -0.975 -0.95 ... -0.025 nan
  * y_rho                (y_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho                (x_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * y_u                  (y_u) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                   ...
    nav_lat_rho          (y_rho, x_rho) float32 2MB dask.array<chunksize=(838, 727), meta=np.ndarray>
    nav_lon_rho          (y_rho, x_rho) float32 2MB dask.array<chunksize=(838, 727), meta=np.ndarray>
    nav_lat_u            (y_u, x_u) float32 2MB dask.array<chunksize=(838, 726), meta=np.ndarray>
    nav_lon_u            (y_u, x_u) float32 2MB dask.array<chunksize=(838, 726), meta=np.ndarray>
    nav_lat_v            (y_v, x_v) float32 2MB dask.array<chunksize=(837, 727), meta=np.ndarray>
    nav_lon_v            (y_v, x_v) float32 2MB dask.array<chunksize=(837, 727), meta=np.ndarray>
Data variables: (12/14)
    hc                   (time_counter) float32 806kB dask.array<chunksize=(1,), meta=np.ndarray>
    theta_s              (time_counter) float32 806kB dask.array<chunksize=(1,), meta=np.ndarray>
    theta_b              (time_counter) float32 806kB dask.array<chunksize=(1,), meta=np.ndarray>
    Tcline               (time_counter) float32 806kB dask.array<chunksize=(1,), meta=np.ndarray>
    Vtransform           (time_counter) float32 806kB dask.array<chunksize=(1,), meta=np.ndarray>
    Cs_r                 (time_counter, s_rho) float32 32MB dask.array<chunksize=(1, 40), meta=np.ndarray>
    ...                   ...
    sc_w                 (time_counter, s_w) float32 33MB dask.array<chunksize=(1, 41), meta=np.ndarray>
    zeta                 (time_counter, y_rho, x_rho) float32 491GB dask.array<chunksize=(1, 838, 727), meta=np.ndarray>
    u                    (time_counter, s_rho, y_u, x_u) float32 20TB dask.array<chunksize=(1, 40, 838, 726), meta=np.ndarray>
    v                    (time_counter, s_rho, y_v, x_v) float32 20TB dask.array<chunksize=(1, 40, 837, 727), meta=np.ndarray>
    temp                 (time_counter, s_rho, y_rho, x_rho) float32 20TB dask.array<chunksize=(1, 40, 838, 727), meta=np.ndarray>
    salt                 (time_counter, s_rho, y_rho, x_rho) float32 20TB dask.array<chunksize=(1, 40, 838, 727), meta=np.ndarray>
Attributes: (12/39)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

## 2. **Select variables** and ensure lon/lat are explicit coordinates

The original dataset contains many variables. For this demo we keep:

- `temp` (temperature)
- `salt` (salinity)
- `zeta` (sea surface height)

We also **load** the 2D longitude/latitude fields and attach them as coordinates (`nav_lon_rho`, `nav_lat_rho`).  
Loading them explicitly avoids repeated remote reads later (plots, masking, regridding, etc.).


In [2]:
ds=ds[['temp','salt','zeta']].assign_coords(
    nav_lon_rho=ds["nav_lon_rho"].load(),
    nav_lat_rho=ds["nav_lat_rho"].load(),
)
ds

/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:3375: UserWarning: Sending large graph of size 329.87 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:3375: UserWarning: Sending large graph of size 329.86 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:3375: UserWarning: Sending large graph of size 329.86 MiB.
This may cause some slowd

<xarray.Dataset> Size: 40TB
Dimensions:       (time_counter: 201600, s_rho: 40, y_rho: 838, x_rho: 727)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2MB 2020-02-01T00:52:30 ... 2...
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * y_rho         (y_rho) float32 3kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (x_rho) float32 3kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    time_instant  (time_counter) datetime64[ns] 2MB dask.array<chunksize=(201600,), meta=np.ndarray>
    nav_lat_rho   (y_rho, x_rho) float32 2MB -1.0 -1.0 -1.0 ... 50.87 50.87
    nav_lon_rho   (y_rho, x_rho) float32 2MB -1.0 -1.0 -1.0 ... 1.653 1.667 1.68
Data variables:
    temp          (time_counter, s_rho, y_rho, x_rho) float32 20TB dask.array<chunksize=(1, 40, 838, 727), meta=np.ndarray>
    salt          (time_counter, s_rho, y_rho, x_rho) float32 20TB dask.array<chunksize=(1, 40, 838, 727), meta=np.ndarray>
    zeta          (time_counter, y_rho, x_rho) float32 491GB dask.array<chunksize=(1, 838, 727), meta=np.ndarray>
Attributes: (12/39)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

## 3. **Build a polygon mask** helper for the model grid
To extract a spatial subset, we load a boundary polygon (GeoJSON) and create a boolean mask on the dataset grid:

- `True`  → grid point is inside the polygon
- `False` → outside

This is useful to reduce the dataset to a Region Of Interest (ROI) before saving or regridding.


In [3]:


def apply_polygon_mask(
    ds: xr.Dataset,
    poly,
    lon_name: str = "nav_lon_rho",
    lat_name: str = "nav_lat_rho",
    mask_name: str = "mask",
) -> xr.Dataset:
    """
    Add a boolean mask to ds that is True where (lon,lat) points fall inside `poly`.

    Parameters
    ----------
    ds : xr.Dataset
        Must contain 2D lon/lat fields (e.g. nav_lon_rho, nav_lat_rho).
    poly : shapely.geometry (Polygon or MultiPolygon)
        In EPSG:4326 lon/lat.
    lon_name, lat_name : str
        Variable names for lon/lat inside ds.
    mask_name : str
        Name of the new coordinate/variable to store the mask.

    Returns
    -------
    xr.Dataset
        Same dataset with a new boolean DataArray `mask_name`
        attached as a coordinate (same dims as lon/lat).
    """
    lon2d = ds[lon_name]
    lat2d = ds[lat_name]

    # numpy arrays
    lon = lon2d.data if isinstance(lon2d.data, np.ndarray) else np.asarray(lon2d.values)
    lat = lat2d.data if isinstance(lat2d.data, np.ndarray) else np.asarray(lat2d.values)

    # --- build mask ---
    try:
        # shapely>=2
        from shapely import contains_xy
        mask_np = contains_xy(poly, lon, lat)
    except Exception:
        # fallback (fast-ish, but only uses exterior ring)
        from matplotlib.path import Path

        def _contains_one_polygon(p):
            x, y = p.exterior.xy
            path = Path(np.column_stack([x, y]))
            pts = np.column_stack([lon.ravel(), lat.ravel()])
            return path.contains_points(pts).reshape(lon.shape)

        # Support MultiPolygon by OR-ing components
        if getattr(poly, "geom_type", None) == "MultiPolygon":
            mask_np = np.zeros(lon.shape, dtype=bool)
            for p in poly.geoms:
                mask_np |= _contains_one_polygon(p)
        else:
            mask_np = _contains_one_polygon(poly)

    # wrap back to DataArray with same dims/coords as lon/lat
    mask_da = xr.DataArray(
        mask_np.astype(bool),
        coords=lon2d.coords,
        dims=lon2d.dims,
        name=mask_name,
    )

    # Attach mask (as a coord, like you were doing)
    return ds.assign_coords({mask_name: mask_da}).where(mask_da)#,drop=True)

    

## 4. **Define the ROI** and apply the mask (load ROI polygon, mask, and subset)

For the regridding demo we focus on a limited area defined by an **outer boundary** polygon (stored in `outer_boundary.geojson`).  
We will:

1. Read the polygon from GeoJSON.
2. Trim the dataset  with polygon mask 
3. Find out which values are 'ground' by  computing not null values of zeta at time_counter=0
4. Stack the spatial coordinate and drop all the ground point 


In [4]:
import geopandas as gpd
# 1. Read the polygon from GeoJSON.

gdf=gpd.read_file("outer_boundary.geojson", driver="GeoJSON")
poly = gdf.geometry.iloc[0]  
#2. Trim the dataset  with polygon mask : zeta_mask
#3. Find out which values are 'ground' by  computing not null values of zeta at time_counter=0 : zeta_mask

zeta_mask= apply_polygon_mask(
    ds.zeta.isel(time_counter=0).compute(),
    poly).notnull()
#zeta_mask
ds_roi = ds
ds_roi['zeta_mask']=zeta_mask

#4. Stack the spatial coordinate and drop all the ground point (ds_in)

ds_roi_1d=ds_roi.stack(point=("y_rho", "x_rho") )

ds_roi_1d = ds_roi_1d.where(ds_roi_1d.zeta_mask,drop=True).drop_vars('zeta_mask')
ds_roi_1d

/srv/conda/envs/notebook/lib/python3.12/site-packages/pyogrio/raw.py:200: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(
/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:3375: UserWarning: Sending large graph of size 332.31 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:3375: UserWarning: Sending large graph of size 331.08 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


<xarray.Dataset> Size: 176GB
Dimensions:       (time_counter: 201600, s_rho: 40, point: 2699)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2MB 2020-02-01T00:52:30 ... 2...
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * point         (point) object 22kB MultiIndex
    time_instant  (time_counter) datetime64[ns] 2MB dask.array<chunksize=(201600,), meta=np.ndarray>
    nav_lat_rho   (point) float32 11kB 46.9 46.91 46.91 ... 47.42 47.43 47.43
    nav_lon_rho   (point) float32 11kB -2.773 -2.867 -2.853 ... -2.613 -2.52
    mask          (point) bool 3kB True True True True ... True True True True
  * y_rho         (point) float32 11kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (point) float32 11kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Data variables:
    temp          (time_counter, s_rho, point) float32 87GB dask.array<chunksize=(1, 40, 2699), meta=np.ndarray>
    salt          (time_counter, s_rho, point) float32 87GB dask.array<chunksize=(1, 40, 2699), meta=np.ndarray>
    zeta          (time_counter, point) float32 2GB dask.array<chunksize=(1, 2699), meta=np.ndarray>
Attributes: (12/39)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [11]:
# Build operator once

def to_healpix(ds_in):
    from regrid_to_healpix.regrid_to_healpix_bilinear import Set
    
    
    lon = ds_in["nav_lon_rho"].values.astype(np.float64)
    lat = ds_in["nav_lat_rho"].values.astype(np.float64)
    
    nr = Set(lon_deg=lon, lat_deg=lat, level=child_level, device="cpu", threshold=0.5, ellipsoid="WGS84")
    cell_ids = np.asarray(nr.get_cell_ids(), dtype=np.int64)
    ncell = int(cell_ids.size)
    
    def to_healpix_point(data_1d):
        out = nr.transform(np.asarray(data_1d, dtype=np.float64), lam=0.1)
        return np.asarray(out, dtype=np.float64)
    
    # Apply to the whole Dataset: only to chosen data_vars
    #vars_to_regrid = ["temp"]  # add "salt", "zeta", ...
    
    
    ds_hp = xr.apply_ufunc(
        to_healpix_point,
        ds_in,
        input_core_dims=[["point"]],
        output_core_dims=[["cell_ids"]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[np.float64],
        dask_gufunc_kwargs={"output_sizes": {"cell_ids": ncell}},
        keep_attrs=True,  # keep dataset + variable attrs where possible
    )
    
    # Re-attach coordinate + its metadata
    ds_hp = ds_hp.assign_coords(cell_ids=("cell_ids", cell_ids))
    ds_hp["cell_ids"].attrs.update({
        "grid_name": "healpix",
        "level": 13,
        "indexing_scheme": "nested",
        "ellipsoid": "WGS84",
    })
    
    
    # compute the child id from the final interest region
    aligned_child_ids = np.unique(healpix_geo.nested.zoom_to(
        parent_ids,
        depth=parent_level,
        new_depth=child_level
    ))
                                  
    # Make sure types match (important: your ds_hp cell_ids look like int64)
    target_ids = aligned_child_ids.astype(ds_hp["cell_ids"].dtype)
    #compute the chunk size 
    chunk_size=4**(child_level - parent_level )
    
    # aline the fill non existing values with np.nan, and take out non interestd zone
    #
    ds_aligned = (
        ds_hp.reindex(cell_ids=target_ids, fill_value=np.nan)
        .chunk({"cell_ids": chunk_size},{"time_counter": time_chunk_size})
    )
    return ds_aligned



## 5.convert to healpix




In [12]:
block = 24   # 48 (or 24*100 etc.)
nt = ds_roi_1d.sizes["time_counter"]
nt = 48 
first = True
for t0 in range(0, nt, block):
    t1 = min(nt, t0 + block)

    ds_in  = to_healpix(ds_roi_1d.isel(time_counter=slice(t0, t1)))

    if first:
        ds_in.to_zarr(zarr_hp_file_path, mode="w", consolidated=False, safe_chunks=False)
        first = False
    else:
        ds_in.to_zarr(
            zarr_hp_file_path,
            mode="a",
            append_dim="time_counter",
            consolidated=False,
            safe_chunks=False,
        )

# consolidate once (optional)
import zarr
zarr.convenience.consolidate_metadata(zarr_hp_file_path)

ModuleNotFoundError: No module named 'regrid_to_healpix'